In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
#read your csvfile(100rows)
category_df=pd.read_csv("URL_CLASSIFICATION.csv")
url=[]
for i in category_df['website_url']:
    url.append(i)

In [3]:
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import os

In [4]:
class ScraperTool:
    def visit_url(self, website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
        content = requests.get(website_url,headers=headers,timeout=20).content
        
        #lxml is apparently faster than other settings.
        soup = BeautifulSoup(content, "lxml")
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "title_tag_content": self.get_html_title_tag(soup),
            "meta_tag_content": self.get_html_meta_tags(soup),
            "headings_content": self.get_html_heading_tags(soup),
            "html_text_content": self.get_text_content(soup)
        }
        #get_tag_count returns a dynamic dictionary. That is why we update the dictionary with a separate update command.
        result.update(self.get_tag_count(soup))
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

    def get_tag_count(self,soup):
        '''returns a dictionary with the frequency of tag for every unique tag found in the page.'''
        tags = soup.find_all()
        return dict(Counter(["tag_count_" + tag.name for tag in tags]))

In [5]:
import spacy as sp
from collections import Counter
sp.prefer_gpu()
import en_core_web_sm
#anconda prompt ko run as adminstrator and copy paste this:python -m spacy download en
nlp = en_core_web_sm.load()
import re
def clean_text(doc):
    '''
    Clean the document. Remove pronouns, stopwords, lemmatize the words and lowercase them
    '''
    doc = nlp(doc)
    tokens = []
    exclusion_list = ["nan"]
    for token in doc:
        if token.is_stop or token.is_punct or token.text.isnumeric() or (token.text.isalnum()==False) or token.text in exclusion_list :
            continue
        token = str(token.lemma_.lower().strip())
        tokens.append(token)
    return " ".join(tokens) 

In [6]:
business=["stock","market","business","finance","financial","economical","analysis","research",
          "jobs","investing","investor","market","marketing","valuation","mortgage","mutual",
          "funds","estate","data","portfolio","entrepreneurship","strategy", "marketing", "company", "employee",
           "startup","payment","work","advertising","build","buy","sell","insurance","salary","consumer","share", "money",
          "investor","buying","management","wealth","legal","opportunity","successful","stipend","expensive",
          "economic","currencies","billion","crisis","unemployment","hospitality","industry","trading",'company',
          "professional", "cheap", "retirement","association","accountants","tax","accounting","accountant","professional",
         "accountancy","finance","manager","corporate","financial","accountability","management","resource","cgfm",'supply',
         "agent","taxation","business",'store','merchantile','services','service','client','consulting','corporation','markets',
         'products','production','export','gift','industrial','consultation','consult','office','exporter','range','cultivator',
         'investment','industries','ship','commerce','credit','aircraft','maitainance','aviation','purchase','shop','distributor',
          'equipment','import','sale','rentals','rental','investors','commercial','retail','workspace','consultation','product',
         'organisation','organization','productivity','freelance','freelancer','serve',"digital","banking","bank","loan",
          "credit","debit",'customer','merchandise','inventory','wholesale','manufacturing','manufacturer',
         "money","saving","savings","economic","investment","home","deposit","corporation","bankunderground","commercial",
          "investing","branch","payment","transaction","overdraft","loans","insurance","investor","industries","industry","company",
         "fund","alliance","stock","invest","price","dividend","broker","department","document","project","mining","mineral",
         "asset","construction","capital","oil","gas","regulations","infrastructure","investors","solar","nextenergy","market",
         "markets","trade","office",'member']
education=["science",'certificate','learn','data','university','skill','code','certification','solutions','maths',
           'developer','learning','professional','career','degree','course','college','education','students','learners',
           'campus','teaching','topics','subjects','teach','academics','teacher','institution','programming',
           'engineering','school','curriculum','graduate','nanodegree','bootcamp','mentor','graduation','assignment',
           'scholarship','class','syllabus','exam','formulas','tutoring','cheatsheet','training',"acadamy","educational",
           "student","study","mba",'admissions','admission','schools','academic','freshman','class','collegiate','careers',
           'examinations','sciences','doctoral','alumni','professor','biosciences','bioengineering','faculty','physcis','grad',
          'programme','chemistry','courses','freshers','history','homeschooling','sat','math','mathematics','cheatsheets',
           'educator','tutorial','tutor','examination','mcqs','lesson','biotech','studies','classroom','literature','poetry',
          'programmes','library','lecture','astronomy','scientific','lab','laboratory','biology','astronomical','encylopaedia',
          'environmental','conservation','physiology','sciencedirect','biochemistry','biophysics','botany','biochemical',
           'scientist','biochemist','wikipedia','telescope','observatory','conference','research','britannica','philosophy',
          'geography','atomic','periodic','element','compound','ecology','ecosystem','journal','chemometrics','pseudoscience',
          'electrochemistry','electrochemical','environment','chemical','concentration','ionic','geochemical','geochemistry',
          'geosciences','geologist','geology','geological','volcano','palaeogeography','algebra','textbook','mathematical',
          'arithmetic','theory','physics','group','mathworld','calculus','amicable','calculator','integer','theorem','universe',
          'gravity','magnet','experiments','quantum',"journals"]
computers_technology=["development","java","algorithms","computer","laptop","technical","programming","visualisation","data",
                      "technology","machines","applications","visuals","apple","engineering","smartphones","array","output",
                      "system","object","display","file","code","execution","error","runtime","debuggung","software","android",
                      "process","approach","statement","language","expression","computer","parsing","run","install","download",
                      "web","internet","update","compare","project","program","result","type","device","byte","application",
                      "programmer","hardware","connection","digital","browser","app","cybersecurity","encode","coding",
                      "technique","tech","algorithm","backtracking","backtrack","javascript","binary","interface","compression",
                      "zip","format","image","pattern","network","developer","desktop","systems","ml","ai","model","mechatronic"
                      ,"machine","modeling","simulation","framework","visualization","modelling","virtual","technologies",
                      "processor","microprocessor","compiler","database","iteration","cloud","compress","developers","graphic",
                      "graphics","visual","microsoft","xml","freeware","e","processing","site","technical","microsoft",
                      "development","unix","repository","hosting","networks","pop3","imap4","disk","memory","cyber",
                      "cryptography"]
health=["health","care","help","life","research","medical","clinic","patient","emergency","safety","rescue","cancer","treatment"
        ,"disease","information","addiction","recovery","clinical","symptom","doctor","performance","medicine","surgery",
        "doctors","specialist","dentist","surgeon","clinics","hospital","telehealth","physiotherapy","consultation","training",
        "transformation","fit","nutrition","gym","athletic","fitness","wellness","healthy","workout","workouts","muscle",
        "weight","body","exercise","calorie","diet","heart","yoga","smoking","tobacco","cigarette","running","marijuana","therapy",
        "drugless","practitioners","reports","allergy","allergies","rehabilitation","deafness","blind","deaf","blindness","mental",
        "deafblind","ayurveda","hiv","aids","bacteria","infection","healthcare","disinfection","disinfectant","accident","broken","strength",
        "membership","yoga","crossfit","pulse","cardio","transplant","wellbeing","psychologist","veterinary","vet","nutritional","mindset",
        "conditioning","detox","liver","kidney","opd"]
food=["plate","chef","meal","delivery","delivered","healthy","milk","dessert","food","delicious","recipe","ingredient","cook",
     "breakfast","lunch","dinner","cooking","taste","prepare","vegetarian","dining","dine","restaurant","menu","order",
      "cuisine","chinese","mexican","beverage","indian","pizza",'drink',"juice","snack","bakery","pantry","flavor","cooked",
      "utensil","knife","delight","roast","dinners","cookware","dinnerware","cups","eat","recipe","recipes","sauce","cocktail",
      "kitchen","serve","scoop","salad","bake","bars","desserts","culinary","butter","buttery","puff","thanksgiving",
      "blend","sweets","dining","tips","cheese","breads","cookies","brownies","candies","creams","dips","dishes","pies",
      "tarts","cookbook","deliciousness","delicious","meals","snacks","seating","capacity","bistro","seafood",
      "wine","dine","menus","celebrate","date","special","taste","authentic","rice","gluten","italian","pastry",
      "shake","indulgance","tasty","sandwich","cafe","espresso","dish","mexican"]
photography=["photograph","photo","image","photography","camera","light","nature","original","inspiring","picture","color",
            "colour","style","edits","creativity","creative","lens","aperture","contrast","natural","candid","sharpening",
            "exposure","gallery","potrait","photographic","portraiture","dark","beauty","portfolio","fashion","photographer",
             "men","women","portraits","landscape","print","exposures","thumbnail","capture","light","contact","photos",
             "modern","background","studio","gallery","collage","editor","editing","edit","capture","video","panorama","view","videos",
             "panaromas","vr","panaromic","images","collection","gigapixels","stereoscopic","click","film","gigapixel","stereographic",
             "share","record","smile","moment","feel","joy","calm","art","creation","theme","customizations","visualizer",
             "pictorial","pixels","expose","album","photographers","template","photoshop","feed","snapshot","composition",
             "focal","aesthetic",'photojournalism','exhibition','digital','journal','500px','photogs','photoshoot','photobooth',
            'transition','creatives','resolution','creatively','picture','collection','magazine']
travel=["experience","stay","flight","hotel","airport","travel","flight","deal","deals","price","destination","visit",
       "airlines","trip","vacation","cruise","resort","holiday","booking","rating","room","city","offer","economy","price",
       "trip","return","international","hostel","fare","airline","airlines","deal","departure","arrival","fly","tickets",
       "book","place","location","date","air","booking","dd","mm","yyyy","status","passenger","boarding",
       "availability","package","tour","visa","trip","accomodation","rental","search","ticket","check","guide","review","coupon",
       "explore","agents","benefits","personal","reserve","property","discount","service","merchandise","register","child","credential",
       "provide","saving","promo","retrieve","mastercard","holidays","view","arrangements","activities","cheap","world",
       "road","suite","guest","travel","outings","venue","reservations","hospitality","client","area","traveler","landmark",
       "luxury","adventure","residances","management","experience","getaway","comfort","geographical","manager","regionanl","traffic",
       "km","board","ticket","visit","drivers","transportation","cities","city","airports","journey","tourist",
       "distance","tourism","ride","drive"]
lawandgovernment=['state','bureau','government','democracy','embassies','ministry','enforcement','law','justice','constitution',
                  'parliament','foreign','defence','heritage','legislature','citizens','passport','visa','citizenship','crime',
                  'immigration','abroad','policy','civil','attorney','legal','police','lawyer','testament','settlement','taxes',
                  'court','district','country','justice','criminal','annexure','advocates','welfare','governance','national','minority',
                  'administration','grievances','minister','resident','chiefs','governmental','authority','cabinet','protocol','election',
                 'forestry','international','pension','portal','central','egovernance','pensions','electoral','voter','parliamentry','constituency',
                 'elector','ministerial','diplomatic','coalition','migration','democratic','citizenship','policies','regulatory','governor','governments',
                 'minorities','commission','enforcement','army','veteran','leader','constitutional','empowerment','amendment',
                 'backward','relief','veterans','pensioner','chief','tenure','inspector','govt','information','royal','departments','suspicious',
                 'affairs','nation','federal','embassy','consulate','congressional','mission','agency','intelligence','legislation','evisa','missions',
                 'enforce','bilateralism','military','authorized','bilateral','officials','prohibit','tax','taxpayer','cops','policing','reform','senate','legislative'
                 ,'council','committee']
streaming_platform=["stream","streaming","shows","movies","online","live","hd","video","premium",'hosting','free','play','album',
                    "rating","web","series","orignal","content","genre","download","watch","music","entertainment","entertain",
                    "language","horror","comedy","romantic","thrill","channel","tv","film","show","episode","playlist",'scene','pantaya',
                    'subscription','theatres','showcase','media','season','feature','award','movie','platform','quality','recommend','podcast','sound',
                    'audio','song','offline','listen','artists','albums','mp3','twitch','access','radio','multistreaming','trial','broadcast']
e_commerce=["shop","shopping","offer","manufacturer","supplier","importer","exporter","discount","men","women","ship",
            "shipping","buy""sell","import","export","mobile","fashion","electronic","equipment","cart","payment","delivery",
            "cash","store","brand","warrenty","supermart","exchange","grocery","product","coupon","deal","stores","price",
            "sale","trend","order","orders","return","gift","e","commerce","customer","design","range","fresh",
            "pay","item","purchase","gift","card","redeem","wholesale","member","retail","apparel","wear","reliable"
            "policy","economical","feedback","pricing","quality","services","Address","emi","accessories","appliances","configure",
            "quality","cost","effective","consumer","collection","brand","credit","paypal","distributor","worldwide","support",
            "buying","manufacture","budget","rating","available","personalized","buyer","recommend","market","season",
            "grow","bestseller","household","track","refund","offers","arrivals","style","freestyle","unisex",
            "color","original","essential","bag","decor","decorations","furniture","distributor","prosumer","standard",
            "western"]
chat_rooms=["chat","room","rooms","meet","people","random","login" ,"send","attach","emoji","emojis","search",
            "video","webcam","community","discussions","group","groups","call","messaging","voice",
             "network","type","typing","talk","sign up","stranger","strangers","second","virtual","date","dating",
            "portal","log","account","photo","video","message","friend","family","share","like","comment",
            "upload","privacy","id","restrict","member","members","whatsapp","facebook","course","join","snapchat",
           "pinterest","discover","roleplay","social","complete","check","tag","archive","temporarily","unavailable","ray",
           "collect","way","bible","prove"]
forums=["forum","forums","login","source","discussion","discussions","guide","post","profanity","offensive",
        "FAQ","terms","policies","real-time","resolved","question","questions","moderators","language",
        "asked", "monthly", "top", "options", "share", "learn","public","answers","private","career","jobs","opportunities",
        "communities", "blog", "acknowledge", "service", "users", "trial", "discovery", "providing", 
        "quickly", "resource", "details", "issues","intense", "release", "interests", 
        "developers", "knowledge", "discuss", "acknowledge", "explore", "campaign", "people", "opinion","reddit","ago","trend",
       "article","ubuntu","theory","start","ars","openforum","eevblog","hub","wattpad","publish","see"]
news=["breaking","latest","headlines","headlines","international","national","news","local","top","stories","times",
      "journalist","coverage","politics","business","updates","current","affairs","livestream","live","today","forecast",
      "newsletter","event","atlantic","vaccine","election","effective","read","minute","review","awards","report",
      "reporter","hollywood","election","donald","trump","pandemic","lockdown","stay","work","editorial","home",
     "interview","celebrity","star","celeb","code","know","bitcoin","big","journal","iwu","block","cast","window","nms",
     "baltic","lithuania","nato","vps","reportedly","sky","b","size","ago","ballon","call","limit","tatafo","seek","commission",
     "crime","box","wealth","credit","muhama","vs","lol","governor","enchantment","abusive"]
games=["game","games","board","strategy","opponent","player","card","play","win","online","multiplayer","role","streaming",
       "gamer","free","racing","rule","action","adventure","fifa","nfl","ea","battlefield","speed","war","legends","arena",
       "match","gaming","gameplay","wars","xbox","playstation","creed","assassin","pc","ps4","warcraft","level","pvp","raid",
       "race","ps3","boardgame","boardgames","move","shoot","chess","mate","championship","hint","rules","tournament","rummy",
      "encounter","guide","club","bridge","coin","keyboard","classic","steering","gun","arcade","wheel","myarcade","gamer",
      "games","game","video","indoor","frame","pro","auction","pinball","repair","pinhedz","machine","hearts","spade","club",
      "diamond","augustgames","lego","battletech","clash","overkill","mortal","mortals","crime","power","destruction",
      "roleplay","character","portal","play","star","simulation","ship","pilot"]
sports=["sport","sporting","fan","tie","chemistry","pass","game","schedule","live","highlight","coverage","player","team","score",
       "stats","premier","standing","fifa","worldcup","football,","goal","futsal","match","standing","basketball",
        "cricket","hockey","golf","athletic","nfl","nba","boxing","scoreboard","playoff","leaderboard","live","watch","ipl",
       "soccer","coach","pro","shoes","season","win","baseball","player","news","football","movement","run","long","injury","midfielder",
       "ready","wwe","wrestlemania","league","mlb","players","stats","leagues","offseason","postseason","location","coach",
       "gymnastics","fitness","gym","physical","ranking","tennis","cricket","athletes","athlete","barcelona","arsenal","chelsea",
       "squad","rise","volleyball","championship","bowling","betting","goal","live","premier","liverpool","champions",
       "stadium","fixture","fifa","messi","fc","badminton","window","fitness","backhand","defence","smash","court",
       "practise","strategies","strength","yonex","workout","singles","strategy","doubles","bodyweight","tip","tips","strike",
       "speed","opponent","level","tennis","soccer","baseball","golf","running","volleyball","swimmming","boxing","tabletennis",
       "skiing","iceskating","rollerskating","rugby","pool","darts","icehockey","hockey","surfing","karate","horseracing","snowboarding",
       "skateboarding","cycling","archery","fishing","gymnastic","sumowrestling","skydiving","bungeejumping","e","state","dibaseball",
       "courtline"]
adult=["porn","pornography","sex","masturbation","bdsm","erotic",'blonde','milf','Hardcore','tits','tit','boobs','pussy','ass',
       'creampie','busty','skinny','fantasy','fantasies','brunette','interracial','hentai','dick','threesome','gangbang','anal',
       'lesbian','gay','masturbate','bondage','doggystyle','nipples','oral','amateur','naked','hookup','cumshot','hairy',
       'fetish','handjobs','webcam','erotica','bisexual','hardcore','milfs','cock',"hd","video","pornhub","free","videos",
      "premium","amateur","fuck","xxx","creampie","cum","teen","milf","pornstar","brazzers","pussy","views","sexy","tits",
      "gay","fucking","bhabhi","black","horny","straight","blonde","blowjob","brunette","rank","anal","porno","bbc","ass",
      "tit","boob","suck","slut","animation","hentai","girl","indian","desi","lesbian","threesome","mom","wife","gangbang",
      "hardcore","dirty","transgender","rough","woman","busty","vporn","massage","hard"]
keywords=streaming_platform+sports+health+games+news+chat_rooms+forums+business+e_commerce+food+education+travel+photography+\
computers_technology+lawandgovernment+adult

In [7]:
pip install flashtext

Note: you may need to restart the kernel to use updated packages.


In [8]:
from flashtext.keyword import KeywordProcessor
kp0=KeywordProcessor()
for word in keywords:
    kp0.add_keyword(word)
kp1=KeywordProcessor()
for word in streaming_platform:
    kp1.add_keyword(word)
kp2=KeywordProcessor()
for word in sports:
    kp2.add_keyword(word)
kp3=KeywordProcessor()
for word in health:
    kp3.add_keyword(word)
kp4=KeywordProcessor()
for word in games:
    kp4.add_keyword(word)
kp5=KeywordProcessor()
for word in news:
    kp5.add_keyword(word)
kp6=KeywordProcessor()
for word in chat_rooms:
    kp6.add_keyword(word)
kp7=KeywordProcessor()
for word in forums:
    kp7.add_keyword(word)
kp8=KeywordProcessor()
for word in business:
    kp8.add_keyword(word)
kp9=KeywordProcessor()
for word in e_commerce:
    kp9.add_keyword(word)
kp10=KeywordProcessor()
for word in food:
    kp10.add_keyword(word)
kp11=KeywordProcessor()
for word in education:
    kp11.add_keyword(word)
kp12=KeywordProcessor()
for word in travel:
    kp12.add_keyword(word)
kp13=KeywordProcessor()
for word in photography:
    kp13.add_keyword(word)
kp14=KeywordProcessor()
for word in computers_technology :
    kp14.add_keyword(word)
kp15=KeywordProcessor()
for word in lawandgovernment :
    kp15.add_keyword(word)
kp16=KeywordProcessor()
for word in adult :
    kp16.add_keyword(word)

In [9]:
def percentage1(dum0,dumx):
    try:
        ans=float(dumx)/float(dum0)
        ans=ans*100
    except:
        return 0
    else:
        return ans
def check_title(x):
    title=df['website_name'].loc[df['cleaned_website_text']==x]
    title=list(title)[0]
    test_list=['forum'] 
    res = [ele for ele in test_list if(ele in title)]
    return res
def check_url(x):
    url=df['website_url'].loc[df['cleaned_website_text']==x]
    url=list(url)[0]
    test_list=['.edu','.ac.'] 
    res = ["Education" for ele in test_list if(ele in url)]
    return res

In [18]:
def find_totalkeywords(x):
    Total_matches=kp0.extract_keywords(x)
    y0=len(Total_matches)
    return y0
def find_category(x):
    global m1,m2,m3,m4,m5
    global Category1,Category2,Category3,Category4,Category5
    y0=find_totalkeywords(x)
    y_1=kp1.extract_keywords(x)
    y1 = len(y_1)
    y_2=kp2.extract_keywords(x)
    y2 = len(y_2)
    y_3=kp3.extract_keywords(x)
    y3 = len(y_3)
    y_4=kp4.extract_keywords(x)
    y4 = len(y_4)
    y_5=kp5.extract_keywords(x)
    y5 = len(y_5)
    y_6=kp6.extract_keywords(x)
    y6 = len(y_6)
    y_7=kp7.extract_keywords(x)
    y7 = len(y_7)
    y_8=kp8.extract_keywords(x)
    y8 = len(y_8)
    y_9=kp9.extract_keywords(x)
    y9 = len(y_9)
    y_10=kp10.extract_keywords(x)
    y10 = len(y_10)
    y_11=kp11.extract_keywords(x)
    y11 = len(y_11)
    y_12=kp12.extract_keywords(x)
    y12 = len(y_12)
    y_13=kp13.extract_keywords(x)
    y13 = len(y_13)
    y_14=kp14.extract_keywords(x)
    y14 = len(y_14)
    y_15=kp15.extract_keywords(x)
    y15 = len(y_15)
    y_16=kp16.extract_keywords(x)
    y16 = len(y_16)
    per1 = float(percentage1(y0,y1))
    per2 = float(percentage1(y0,y2))
    per3 = float(percentage1(y0,y3))
    per4 = float(percentage1(y0,y4))
    per5 = float(percentage1(y0,y5))
    per6 = float(percentage1(y0,y6))
    per7 = float(percentage1(y0,y7))
    per8 = float(percentage1(y0,y8))
    per9 = float(percentage1(y0,y9))
    per10 = float(percentage1(y0,y10))
    per11 = float(percentage1(y0,y11))
    per12 = float(percentage1(y0,y12))
    per13 = float(percentage1(y0,y13))
    per14 = float(percentage1(y0,y14))
    per15 = float(percentage1(y0,y15))
    per16 = float(percentage1(y0,y16))
    check_forum=check_title(x)
    if check_forum:
        per7=100.00
    check_edu=check_url(x)
    if check_edu:
        per11=100.0
    if y0==0:
        Category1='None'
        Category2='None'
        Category3='None'
        Category4='None'
        Category5='None'
        m1=0.0
        m2=0.0
        m3=0.0
        m4=0.0
        m5=0.0
        matched_catkeywords='None'
    else:
        per={per1:"Streaming Services",per2:"Sports",per3:"Health and Fitness",per4:"Games",
             per5:"News",per6:"Social Networking and Messaging",per7:"Forums",per8:"Business/Corporate",per9:"E-Commerce",
             per10:"Food",per11:"Education",per12:"Travel",per13:"Photography",per14:"Computers and Technology",
             per15:"Law and Government",per16:"Adult"}
        
        all_keys=per.keys()
        m1=max(all_keys)
        Category1=per[m1]
        del per[m1]
        if per:
            all_keys=per.keys()
            m2=max(all_keys)
            Category2=per[m2]
            del per[m2]
        else:
            m2=0.0
            Category2='None'
        if per:
            all_keys=per.keys()
            m3=max(all_keys)
            Category3=per[m3]
            del per[m3]
        else:
            m3=0.0
            Category3='None'
        if per:
            all_keys=per.keys()
            m4=max(all_keys)
            Category4=per[m4]
            del per[m4]
        else:
            m4=0.0
            Category4='None'
        if per:
            all_keys=per.keys()
            m5=max(all_keys)
            Category5=per[m5]
        else:
            m5=0.0
            Category5='None'

        keywords={"Streaming Services":y_1,"Sports":y_2,"Health and Fitness":y_3,"Games":y_4,"News":y_5,"Social Networking and Messaging":y_6,"Forums":y_7
                  ,"Business/Corporate":y_8,"E-Commerce":y_9,"Food":y_10,"Education":y_11,"Travel":y_12,"Photography":y_13,
                  "Computers and Technology":y_14,"Law and Government":y_15,"Adult":y_16}
        matched_catkeywords=keywords[Category1]
    return y0,matched_catkeywords,Category1,m1,Category2,m2,Category3,m3,Category4,m4,Category5,m5

In [12]:
scraperTool = ScraperTool()
urls= url
series_list = []
for url in urls:
    try:
        series_list.append(scraperTool.visit_url(url))
    except:
        continue
df=pd.DataFrame(pd.concat(series_list,axis=1,sort=False)).T
columns_to_combine = ["title_tag_content","meta_tag_content","headings_content","html_text_content"]
df["website_text"] =df.apply(lambda row: " ".join(row[columns_to_combine].astype(str)),axis=1)
df = df[["website_url","website_name","website_text"]]
df["cleaned_website_text"]  = df.apply(lambda row: clean_text(row["website_text"]) ,axis=1)
df = df.drop(["website_text"],axis=1)
df

,website_url,website_name,cleaned_website_text
0,https://www.booking.com/index.html?aid=1743217,booking,official site good hotel accommodation big sav...
1,https://travelsites.com/expedia/,travelsites,expedia hotel book sites like use vacation wor...
2,https://travelsites.com/tripadvisor/,travelsites,tripadvisor hotel book sites like previously d...
3,https://www.momondo.in/?ispredir=true,momondo,cheap flights search compare flights momondo f...
4,https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...,ebookers,bot create free account create free account si...
5,https://book.priceline.com/?refid=8431&refclic...,priceline,hotel reservations bundle save book flight sal...
6,https://www.trip.com/index?locale=en_us&allian...,trip,official travel deals promotion choose million...
7,https://www.orbitz.com/,orbitz,bot create free account create free account si...
8,https://www.travelocity.com/,travelocity,bot create free account create free account si...
9,https://www.hotwire.com/,hotwire,cheap hotels cars flights minute travel deals ...


In [19]:
Total=[]
match=[]
Cat1=[]
percent1=[]
Cat2=[]
percent2=[]
Cat3=[]
percent3=[]
Cat4=[]
percent4=[]
Cat5=[]
percent5=[]
final={}
for i in df['cleaned_website_text']:
    Total.append(find_category(i)[0])
    match.append(find_category(i)[1])
    Cat1.append(find_category(i)[2])
    percent1.append(find_category(i)[3])
    Cat2.append(find_category(i)[4])
    percent2.append(find_category(i)[5])
    Cat3.append(find_category(i)[6])
    percent3.append(find_category(i)[7])
    Cat4.append(find_category(i)[8])
    percent4.append(find_category(i)[9])
    Cat5.append(find_category(i)[10])
    percent5.append(find_category(i)[11])
    
final['total_matched']=Total
final['matched_category_keywords']=match
final['Category1']=Cat1
final['percent1']=percent1
final['Category2']=Cat2
final['percent2']=percent2
final['Category3']=Cat3
final['percent3']=percent3
final['Category4']=Cat4
final['percent4']=percent4
final['Category5']=Cat5
final['percent5']=percent5
new=pd.DataFrame(final)
df=df.join(new)
df

,website_url,website_name,cleaned_website_text,total_matched,matched_category_keywords,Category1,percent1,Category2,percent2,Category3,percent3,Category4,percent4,Category5,percent5
0,https://www.booking.com/index.html?aid=1743217,booking,official site good hotel accommodation big sav...,277,"[hotel, saving, hotel, destination, hotel, rev...",Travel,63.537906,News,15.523466,Business/Corporate,14.801444,Social Networking and Messaging,11.191336,E-Commerce,9.386282
1,https://travelsites.com/expedia/,travelsites,expedia hotel book sites like use vacation wor...,304,"[hotel, book, vacation, city, offer, fare, hot...",Travel,42.763158,E-Commerce,16.118421,Computers and Technology,15.460526,Business/Corporate,15.131579,Social Networking and Messaging,10.526316
2,https://travelsites.com/tripadvisor/,travelsites,tripadvisor hotel book sites like previously d...,114,"[hotel, book, deal, review, offer, hotel, hote...",Travel,50.000000,Social Networking and Messaging,15.789474,Photography,13.157895,Computers and Technology,12.280702,News,11.403509
3,https://www.momondo.in/?ispredir=true,momondo,cheap flights search compare flights momondo f...,371,"[cheap, search, cheap, flight, fare, airline, ...",Travel,74.123989,Business/Corporate,14.824798,E-Commerce,13.746631,Computers and Technology,7.816712,Social Networking and Messaging,7.277628
4,https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...,ebookers,bot create free account create free account si...,196,"[booking, travel, travel, flight, hotel, holid...",Travel,32.142857,Social Networking and Messaging,20.918367,E-Commerce,12.755102,News,12.244898,Business/Corporate,8.673469
5,https://book.priceline.com/?refid=8431&refclic...,priceline,hotel reservations bundle save book flight sal...,152,"[hotel, reservations, book, flight, hotel, sav...",Travel,69.736842,E-Commerce,18.421053,Business/Corporate,17.105263,Social Networking and Messaging,6.578947,Forums,5.263158
6,https://www.trip.com/index?locale=en_us&allian...,trip,official travel deals promotion choose million...,69,"[travel, deals, hotel, guest, review, book, fl...",Travel,53.623188,E-Commerce,17.391304,Business/Corporate,13.043478,Computers and Technology,8.695652,Social Networking and Messaging,7.246377
7,https://www.orbitz.com/,orbitz,bot create free account create free account si...,156,"[booking, travel, travel, vacation, package, v...",Travel,30.769231,E-Commerce,22.435897,Business/Corporate,14.102564,News,9.615385,Sports,7.051282
8,https://www.travelocity.com/,travelocity,bot create free account create free account si...,150,"[booking, travel, hotel, trip, travel, vacatio...",Travel,32.666667,Social Networking and Messaging,27.333333,E-Commerce,20.000000,Business/Corporate,10.666667,News,7.333333
9,https://www.hotwire.com/,hotwire,cheap hotels cars flights minute travel deals ...,10,"[cheap, travel, deals, deal, cheap, hotel, fli...",Travel,90.000000,Business/Corporate,20.000000,E-Commerce,10.000000,Adult,0.000000,None,0.000000


In [ ]:
z=dict()
a=dict()
b=dict()
c=dict()
d=dict()
e=dict()
f=dict()
g=dict()
h=dict()
i=dict()
j=dict()
k=dict()
l=dict()
m=dict()
n=dict()
o=dict()
words = df["cleaned_website_text"].loc[90].split()
for word in words:
    word=word.lower()
    if word[-1]==',' :
      word=word[:-1]
    if word in streaming_platform:
        if word in z:
          z[word] = z[word] + 1
        else:
          z[word] = 1

    if word in sports:
        if word in a:
          a[word] = a[word] + 1
        else:
          a[word] = 1
    
    if word in health:
        if word in b:
          b[word] = b[word] + 1
        else:
          b[word] = 1
        
    if word in games:
        if word in c:
          c[word] = c[word] +1
        else:
          c[word] = 1

    if word in news:
        if word in d:
          d[word] = d[word] +1
        else:
          d[word] = 1

    if word in chat_rooms:
          if word in e:
            e[word] = e[word] +1
          else:
            e[word] = 1

    if word in forums:
          if word in f:
            f[word] = f[word] +1
          else:
            f[word] = 1

    if word in business:
          if word in g:
            g[word] = g[word] +1
          else:
            g[word] = 1

    if word in e_commerce:
          if word in h:
            h[word] = h[word] +1
          else:
            h[word] = 1

    if word in education:
          if word in i:
            i[word] = i[word] +1
          else:
            i[word] = 1

    if word in lawandgovernment:
          if word in j:
            j[word] = j[word] +1
          else:
            j[word] = 1

    if word in computers_technology:
          if word in k:
            k[word] = k[word] +1
          else:
            k[word] = 1

    if word in photography:
          if word in l:
            l[word] = l[word] +1
          else:
            l[word] = 1

    if word in food:
          if word in m:
            m[word] = m[word] +1
          else:
            m[word] = 1

    if word in travel:
          if word in n:
            n[word] = n[word] +1
          else:
            n[word] = 1
    if word in adult:
          if word in o:
            o[word] = o[word] +1
          else:
            o[word] = 1

print("Streaming key words")
for w in sorted(z, key=z.get, reverse=True):
    print(w, z[w])
print("total streaming key words are :",sum(z.values()))

print("\n\n")

print("Sports key words")
for w in sorted(a, key=a.get, reverse=True):
    print(w, a[w])
print("total sports key words are :",sum(a.values()))

print("\n\n")

print("health key words")
for w in sorted(b, key=b.get, reverse=True):
    print(w, b[w])
print("total health key words are :",sum(b.values()))

print("\n\n")

print("games key words")
for w in sorted(c, key=c.get, reverse=True):
    print(w, c[w])
print("total games key words are :",sum(c.values()))

print("\n\n")

print("news key words")
for w in sorted(d, key=d.get, reverse=True):
    print(w, d[w])
print("total news key words are :",sum(d.values()))

print("\n\n")

print("chatrooms key words")
for w in sorted(e, key=e.get, reverse=True):
    print(w, e[w])
print("total chatroom key words are :",sum(e.values()))

print("\n\n")

print("forums key words")
for w in sorted(f, key=f.get, reverse=True):
    print(w, f[w])
print("total forum key words are :",sum(f.values()))

print("\n\n")

print("business key words")
for w in sorted(g, key=g.get, reverse=True):
    print(w, g[w])
print("total business key words are :",sum(g.values()))

print("\n\n")

print("e_commerce key words")
for w in sorted(h, key=h.get, reverse=True):
    print(w, h[w])
print("total e_commerce key words are :",sum(h.values()))

print("\n\n")

print("education key words")
for w in sorted(i, key=i.get, reverse=True):
    print(w, i[w])
print("total education key words are :",sum(i.values()))

print("\n\n")

print("law and government key words")
for w in sorted(j, key=j.get, reverse=True):
    print(w, j[w])
print("total law and government key words are :",sum(j.values()))

print("\n\n")

print("technology key words")
for w in sorted(k, key=k.get, reverse=True):
    print(w, k[w])
print("total technology key words are :",sum(k.values()))

print("\n\n")

print("photography key words")
for w in sorted(l, key=l.get, reverse=True):
    print(w, l[w])
print("total photgraphy key words are :",sum(l.values()))

print("\n\n")

print("food key words")
for w in sorted(m, key=m.get, reverse=True):
    print(w, m[w])
print("total technology key words are :",sum(m.values()))

print("\n\n")

print("travel key words")
for w in sorted(n, key=n.get, reverse=True):
    print(w, n[w])
print("total photgraphy key words are :",sum(n.values()))

print("\n\n")

print("adult key words")
for w in sorted(o, key=o.get, reverse=True):
    print(w, o[w])
print("total adult key words are :",sum(o.values()))

In [ ]:
d = dict()

words = df["cleaned_website_text"].loc[90].split()
for word in words:
    word=word.lower()
    if word in d:
          d[word] = d[word] + 1
    else:
          d[word] = 1

for w in sorted(d, key=d.get, reverse=True):
    print(w, d[w])

In [ ]:
#df.drop([0,10,90],inplace=True)
#df=df.reset_index(drop=True)

In [ ]:
#final csv(above100)
df.to_csv("FINAL_DATASET.csv")